# 1. milvus

In [7]:
import langchain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

dir(langchain.document_loaders)

['Any',
 'LangChainDeprecationWarning',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_old_to_new_name',
 'is_interactive_env',
 'warnings']

In [19]:
from langchain_community.document_loaders import TextLoader
import os

all_splits = []
f1 = 'textdata/'
for f2 in os.listdir(f1):
    f3 = os.path.join(f1,f2)
    tld = TextLoader(f3)
    doc = tld.load()
    sp = text_splitter.split_documents(doc)
    all_splits+=sp
    
all_splits = all_splits[:100] # 

print(len(all_splits))


100


In [20]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(model_name='embedding/bge/bge-m3/')


In [21]:
from langchain.vectorstores.milvus import Milvus

URI = 'http://localhost:19530'
connection_args = { 'uri': URI }

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name='sandboxllm',
    drop_old=True,
).from_documents(
    all_splits,
    embedding=embeddings,
    collection_name='sandboxllm',
    connection_args=connection_args,
)

In [22]:
query = "What is APT ?"
docs = vector_store.similarity_search(query)

print(len(docs))



4


# 2. RAG Workflow

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(
    streaming=True,
    verbose=True,
    openai_api_key='none',
    openai_api_base='http://localhost:8000/v1',
    model_name='ChatGLM'
)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough


retriever = vector_store.as_retriever()

template = """Below context is the sandbox analysis result. Please answer the question at the end based on the context. 
{context}
Question: {question}
Answer:"""
rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

print(rag_chain.invoke("What is the malicious behaviors for sample xxx?"))
